# Project 3: House Price Prediction

Objective: Predict house prices based on various features like location, size, and house
characteristics.

### Data Cleaning: Handle missing values, preprocess data (e.g., encoding categorical variables, scaling features), and remove or treat outliers

## Imports

In [73]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Data

We will open the .csv file to perform exploratory data analysis (EDA) to understand the dataset and
visualize trends.

In [74]:
df = pd.read_csv("../data/house_prices.csv")

In [75]:
df.head()

,Id,Dwell_Type,Zone_Class,LotFrontage,LotArea,Road_Type,Alley,Property_Shape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Property_Sale_Price
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## Dealing with Outliers

In [76]:
def handle_outlier(col,df):
    q1=df[col].quantile(0.25)
    q3=df[col].quantile(0.75)
    IQR=q3-q1

    lower_b=q1-1.5*IQR
    upper_b=q3+1.5*IQR

    for i in range(len(df)):
        if df.loc[i,col]>upper_b :
            df.loc[i,col]=upper_b
        elif df.loc[i,col]<lower_b:
            df.loc[i,col]=lower_b


handle_outlier('OverallQual',df)
handle_outlier('GrLivArea',df)

/tmp/ipykernel_3605/1157443851.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2747.625' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i,col]=upper_b


In [77]:
df[(df['SaleCondition'].isin(['Normal','Abnorml'])) & (df['Property_Sale_Price']>700000)]

,Id,Dwell_Type,Zone_Class,LotFrontage,LotArea,Road_Type,Alley,Property_Shape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Property_Sale_Price
691,692,60,RL,104.0,21535,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,1,2007,WD,Normal,755000
1182,1183,60,RL,160.0,15623,Pave,NaN,IR1,Lvl,AllPub,...,555,Ex,MnPrv,NaN,0,7,2007,WD,Abnorml,745000


In [78]:
df = df.drop(index=[691,1182])

# Fill NAN Values - Categorical Data

In [79]:
len(df)

1458

## Fill NAN for basement
- **BsmtCond**: Condition of the basement (a rating).
- **BsmtQual**: Quality of the basement (a rating).
- **BsmtFinType1**: Type of finished basement (e.g., rec room, living space).
- **BsmtExposure:** Exposure level of the basement (e.g., walkout, garden level).
- **BsmtFinType2**: Type of secondary finished basement (if applicable).

In [80]:
bsmt_str_cols =  ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']
df[bsmt_str_cols] = df[bsmt_str_cols].fillna('None')

In [81]:
df[['BsmtCond', 'BsmtQual', 'BsmtFinType1', 'BsmtExposure', 'BsmtFinType2']][df['BsmtCond'].isnull()]

,BsmtCond,BsmtQual,BsmtFinType1,BsmtExposure,BsmtFinType2


## Fill NAN for garage
- **GarageQual**: Quality of the garage (a rating).
- **GarageCond**: Condition of the garage (a rating).
- **GarageFinish**: Interior finish of the garage (e.g., unfinished, finished).
- **GarageType**: Type of garage (e.g., attached, detached).

In [59]:
gar_str_cols = ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']
df[gar_str_cols] = df[gar_str_cols].fillna('None')

In [60]:
df[['GarageQual', 'GarageCond', 'GarageFinish', 'GarageType']][df['GarageQual'].isnull()]

,GarageQual,GarageCond,GarageFinish,GarageType


## Dealing with NAN in MasVnrType

In [61]:
df["MasVnrType"] = df["MasVnrType"].fillna("None")

In [62]:
df[['MasVnrType']].isnull().sum()

MasVnrType    0
dtype: int64

## Dealing with NAN in FireplaceQu

In [63]:
df["FireplaceQu"] = df["FireplaceQu"].fillna("None")

In [64]:
df[['FireplaceQu']].isnull().sum()

FireplaceQu    0
dtype: int64

In [65]:
df = df.dropna(axis=0,subset= ['Electrical'])

# Fill NAN Values - Numeric Data

## Dealing with NAN in GarageYrBlt

In [66]:
df['GarageYrBlt'] = df['GarageYrBlt'].fillna(0)

## Dealing with NAN in LotFrontage

In [67]:
df['LotFrontage'] = df.groupby('Neighborhood')['LotFrontage'].transform(lambda val: val.fillna(val.mean()))

## Dealing with NAN in MasVnrArea

In [68]:
df["MasVnrArea"] = df["MasVnrArea"].fillna(0)

In [69]:
df[['MasVnrType', 'MasVnrArea']][(df['MasVnrArea'] == 0) & (df['MasVnrType'] != "None")]

,MasVnrType,MasVnrArea
688,BrkFace,0.0
1241,Stone,0.0


In [70]:
df.groupby('MasVnrType')['MasVnrArea'].mean()

MasVnrType
BrkCmn     247.666667
BrkFace    259.008989
None         1.086108
Stone      239.304688
Name: MasVnrArea, dtype: float64

In [71]:
df.loc[688, 'MasVnrArea'] = 259.008989
df.loc[1241, 'MasVnrArea'] = 239.304688


## Drop Columns

In [ ]:
df = df.drop(['Id', 'PoolQC', 'MiscFeature', 'Alley', 'Fence'], axis=1)

In [ ]:
df.columns

# Trandsform Data

## Use log Transform on Property_Sale_Price for better distribution

In [29]:
df['Property_Sale_Price_natural_log'] = np.log(df['Property_Sale_Price'])

/tmp/ipykernel_8496/2118678732.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Property_Sale_Price_natural_log'] = np.log(df['Property_Sale_Price'])


## Add new column contains Grage Age

In [45]:
df['GarageAge'] = 2024 - df['GarageYrBlt']

# Display Data

In [51]:
df.head()

,Dwell_Type,Zone_Class,LotFrontage,LotArea,Road_Type,Property_Shape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Property_Sale_Price
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000


## Encoding and Scaling Features

In [57]:
df.head()

,Dwell_Type,Zone_Class,LotFrontage,LotArea,Road_Type,Property_Shape,LandContour,Utilities,LotConfig,LandSlope,...,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Property_Sale_Price,Property_Sale_Price_natural_log
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,2,2008,WD,Normal,208500,12.247694
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,5,2007,WD,Normal,181500,12.109011
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,9,2008,WD,Normal,223500,12.317167
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,0,0,0,0,2,2006,WD,Abnorml,140000,11.849398
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,12,2008,WD,Normal,250000,12.429216


In [58]:
pd.get_dummies(df, dtype='int')

,Dwell_Type,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,0,0,1,0,0,0,0,1,0
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,0,0,1,0,0,0,0,1,0
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,0,0,1,0,0,0,0,1,0
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,0,0,1,1,0,0,0,0,0
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,62.0,7917,6,5,1999,2000,0.0,0,0,...,0,0,0,1,0,0,0,0,1,0
1456,20,85.0,13175,6,6,1978,1988,119.0,790,163,...,0,0,0,1,0,0,0,0,1,0
1457,70,66.0,9042,7,9,1941,2006,0.0,275,0,...,0,0,0,1,0,0,0,0,1,0
1458,20,68.0,9717,5,6,1950,1996,0.0,49,1029,...,0,0,0,1,0,0,0,0,1,0


## Export Cleaned Data to CSV File

In [52]:
df.to_csv('cleaned_house_data.csv')

----